## 4. Model Evaluation
In machine learning, model validation is referred to as the process where a trained model is evaluated with a testing data set. The testing data set is a separate portion of the same data set from which the training set is derived. The main purpose of using the testing data set is to test the generalization ability of a trained model ([Alpaydin 2010](http://scholar.google.com/scholar_lookup?title=Introduction%20to%20machine%20learning&author=E.%20Alpaydin&publication_year=2010)).
 
 In simpler terms, model evaluation tells you what kind of performance you can expect once your run **future** data through the model.
 
 The outline of the model building and evaluation process is given below.
 <img src="../img/4/model_evaluation_process.PNG" width=400>
 
 We will start by training a classification model that uses the [scikit-learn implementation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) of the [Random Forest](https://en.wikipedia.org/wiki/Random_forest) algorithm.

In [ ]:
# load necessary Python packages
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, TimeSeriesSplit
from sklearn.metrics import average_precision_score, accuracy_score
from sklearn.metrics import precision_recall_curve, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
%matplotlib inline

# data are stored here
data_folder = '../datasets'

# integer for the random seed
THE_ANSWER = 42

In [ ]:
# load all data
data = np.load('%s/data.npz' % data_folder)

# extract training data
X_train, y_train = data['X_train'], data['y_train']

The *X_train* 2-d array 33 contains features (columns) that describe objects of interest. Based on my experience, about 95% of project time is spent writing code to create this array.

In [ ]:
print(X_train)
print(X_train.shape)

In [ ]:
# let's take a look at the class labels stored in the y_train array
print(y_train)

# only 34% of objects are of class 1 --> the data set is imbalanced
print('The class == 1 objects make %0.f%% of the training set.' % (100*y_train.sum()/y_train.size))

In [ ]:
# instantiate the RandomForestClassifier
model = RandomForestClassifier()

# print out the default values of model hyper-parameters (will talk about hyper-parameters later)
model

In [ ]:
# set the number of trees to 500, and the random seed to THE_ANSWER for reproducibility
model.set_params(n_estimators=500, random_state=THE_ANSWER)
#model = RandomForestClassifier(n_estimators=500, random_state=THE_ANSWER) # another way of doing it

# check that the model changed
model

In [ ]:
# some values in the training set are NaN (i.e., are missing).
# The Random Forest classifier does not like that, so we set NaN values to 10000.
X_train_fixed = X_train.copy()
X_train_fixed[np.isnan(X_train_fixed)] = 10000

In [ ]:
# fit the model using training data
model.fit(X_train_fixed, y_train)

## Classification metrics

### Accuracy

Now that we have fit a model, we need to evaluate its performance.
That can be done using various [classification metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics), but the simplest one is accuracy.
Accuracy is defined as the percentage of correct predictions for the test data.

$$accuracy = \frac{{\rm number\, of\, correct\, predictions}}{{\rm number\, of\, all\, predictions}}$$

In [ ]:
# first, extract test data
X_test, y_test = data['X_test'], data['y_test']

# if there are NaNs in the test set, set them to 10000.
X_test_fixed = X_test.copy()
X_test_fixed[np.isnan(X_test)] = 10000

# only 34% of objects are of class 1 --> the data set is imbalanced
print('The class == 1 objects make %0.f%% of the test set.' % (100*y_test.sum()/y_test.size))

# now push the test set through the model to get the predicted classes
y_pred = model.predict(X_test_fixed)

print('Predicted class:', y_pred)
print('True class:     ', y_test)

In [ ]:
# calculate the accuracy
print('The accuracy is %.3f.' % accuracy_score(y_test, y_pred))

That's a great result, right? Maybe not since we have an imbalanced data set (i.e., class == 1 objects make only 35% of the test set). Here's an extreme example how accuracy can be misleading when the classes are imbalanced:

Imagine a data set where 90% of objects are of class 0. A naive classifier that predicts 0 for all objects will have an accuracy of 90%!

### Precision and Recall

In my opinion, what most business people would like to know is the following:
1. What is the detection rate? Did we correctly identify 50%, 80%, or more of all class 1 objects (e.g., instructions that will fail to settle)?
2. What is the false alarm rate? What percentage of those tagged as class 1 are **not** of class 1?

The detection rate is called ["recall"](https://en.wikipedia.org/wiki/Precision_and_recall) in machine learning, and the false alarm rate is related to ["precision"](https://en.wikipedia.org/wiki/Precision_and_recall) (actually, it is 1-precision).

In [ ]:
# GENIA: use graphics on this page to explain the confusion matrix, precision, and recall
# https://www.jeremyjordan.me/evaluating-a-machine-learning-model/

Now that we understand the confusion matrix, precision, and recall, let's calculate them using the test set for the model we just trained.

In [ ]:
p = precision_score(y_test, y_pred)
r = recall_score(y_test, y_pred)
print('The precision obtained on the test set is %.0f%% and recall is %.0f%%.' % (p*100, r*100))

The above precision and recall indicate that our model is able to detect 83% of all class == 1 objects (e.g., failed settlements), while having a false alarm rate of only 14% (remember that the false alarm rate = 1 - precision).

But what if a detection rate of 86% is not good enough for your use case? What if you willing to accept a higher false alarm rate if that would enable you to identify more of class == 1 objects (i.e., have a higher detection rate)?

This can be achieved by:
1. Calculating classification **scores** (values 0 **to** 1) instead of predicting class **labels** (values 0 **or** 1),
2. Plotting the precision-recall curve, and
3. Selecting all objects that have the classification score higher than some threshold.

Let's start by calculating and explaining classification scores.

In [ ]:
# push the test data set through the model to get the classification *scores* (not labels)
# (note that we use the "predict_proba" method and not "predict")
y_scores = model.predict_proba(X_test_fixed)

# the model calculates the score for both classes (0 and 1). Each row sums to 1.
print(y_scores)
print('The y_scores is a 2-dimensional array:', y_scores.shape)

Now that we have classification scores, let's plot the (1-precision) vs. recall curve.

In [ ]:
# plot the detection rate vs. false alarm rate
p, r, thresh = precision_recall_curve(y_test, y_scores[:, 1])
thresh = np.append(0, thresh)
print('Score > %.3f:  %.0f%% detection rate at 10%% false alarm rate.' % (thresh[(1-p) > 0.1][-1], 100*r[(1-p) > 0.1][-1]))
print('Score > %.3f:  %.0f%% detection rate at 15%% false alarm rate.' % (thresh[(1-p) > 0.15][-1], 100*r[(1-p) > 0.15][-1]))
print('Score > %.3f:  %.0f%% detection rate at 20%% false alarm rate.' % (thresh[(1-p) > 0.2][-1], 100*r[(1-p) > 0.2][-1]))
print('Score > 0.000: 100%% detection rate at %.0f%% false alarm rate (naive approach).' % (100-100*y_test.sum()/y_test.size))

fig, ax = plt.subplots()
ax.step((1-p)*100, r*100, color='b', where='post', label='late settlements')
ax.set_xlabel('False alarm rate [%]')
ax.set_ylabel('Detection rate [%]')
ax.set_ylim([0.0, 101])
_ = ax.set_xlim([0.0, 100])

Thus, if we select all objects (rows) that have the classification score (y_scores[:, 1]) greater than 0.557, we can expect a sample where 78% of objects will be of class 1 (e.g., will be failed settlements), and only 10% will be of class 0.

Example: Let's say there are 100 objects (rows) that have the classification score greater than 0.577. This mean that 90 objects will truly be of class 1 (false alarm rate is 10% --> precision is 90% --> 100*0.9 = 90). The total number of class 1 objects is 115 ( = 90/0.78 --> because detection rate is 78%), which means the algorithm misses 25 objects (rows) of class 1.

The area under the precision-recall curve (AUPRC) is also a [metric](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html). The perfect classifier (model) has a detection rate of 100% and false alarm rate of 0%, and the AUPRC = 1.

In [ ]:
# calculate the area under the precision-recall curve for class == 1 objects
# by feeding the true class label (y_test) and the predicted classification scores (y_scores[:, 1])
print('The area under the precision-recall curve is %.3f.' % average_precision_score(y_test, y_scores[:, 1]))

### Log loss or cross-entropy

The metric of choice for evaluating multi-class classification models. For more details, see [here](http://wiki.fast.ai/index.php/Log_Loss) and [here](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.log_loss.html#sklearn.metrics.log_loss).

## Regression metrics

**(Root) Mean Squared Error** is simply defined as the (root of the) average of squared differences between the predicted output and the true output. A smaller value of this metric indicates a better model. The root of MSE is in units of *y*.
$$MSE(y_{true}, y_{pred})=\frac{1}{n_{samples}}\sum{(y_{true} - y_{pred})^2}$$

[${\bf R^2}$ **coefficient**](https://en.wikipedia.org/wiki/Coefficient_of_determination) or the **coefficient of determination** represents the proportion of variance in the outcome that our model is capable of predicting based on its features. The values range from 1 (perfect model) to minus infinity (the model can be infinitely bad).
$$R^2(y_{true}, y_{pred}) = 1 - \frac{\sum{(y_{true} - y_{pred})^2}}{\sum{(y_{true} - \bar{y})^2}}; \bar{y} = \frac{1}{n_{samples}}\sum{y_{true}}$$

## Strategies for splitting data into train, validation, and test sets

In [ ]:
cv_scores = cross_val_score(model, X_train_fixed, y_train, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=THE_ANSWER), n_jobs=4, scoring='average_precision')
print('Average CV score is %.4f, and the rms scatter is %.4f' % (np.average(cv_scores), np.std(cv_scores, ddof=1)))

In [ ]:
X_valid, y_valid = data['X_valid'], data['y_valid']
X_devel, y_devel = data['X_devel'], data['y_devel']
X_test, y_test = data['X_test'], data['y_test']